In [1]:
import pandas as pd
import numpy as np
import statsmodels.tsa.stattools as ts
from datetime import datetime, timedelta
import os

In [ ]:
directories = ["Binance_Historical_1m_FUTURES_2_days_2022-06-07T21:34:10", "Binance_Historical_1m_SPOT_2_days_2022-06-07T21:35:14"]

In [ ]:
# MERGING INTERSECTION
spot = []
margin = []
with os.scandir('raw_data/%s' % directories[0]) as entries:
  for entry in entries:
    entry_arr = entry.name.split("_")
    # instrument = "_".join(entry_arr[0:2])
    margin.append(entry_arr[0])
        
with os.scandir('raw_data/%s' % directories[1]) as entries:
    for entry in entries:
      entry_arr = entry.name.split("_")
      # instrument = "_".join(entry_arr[0:2])
      spot.append(entry_arr[0])

intersection = set(spot).intersection(margin)

df_closings = pd.DataFrame()
with os.scandir('raw_data/%s' % directories[0]) as entries:
    for idx, entry in enumerate(entries):
        if entry.name.split("_")[0] in intersection:
            instrument = "_".join(entry.name.split("_")[0:2])
            df = pd.read_csv('raw_data/%s/%s' % (directories[0], entry.name), index_col="Date")
            df = df[["Close"]].copy()
            df.columns = [instrument]
            df_closings = pd.concat([df_closings, df], axis=1)
        
with os.scandir('raw_data/%s' % directories[1]) as entries:
    for idx, entry in enumerate(entries):
        if entry.name.split("_")[0] in intersection:
            instrument = "_".join(entry.name.split("_")[0:2])
            df = pd.read_csv('raw_data/%s/%s' % (directories[1], entry.name), index_col="Date")
            df = df[["Close"]].copy()
            df.columns = [instrument]
            df_closings = pd.concat([df_closings, df], axis=1)
        
df_closings.index = pd.to_datetime(df_closings.index)

df_closings.fillna(method="ffill", inplace=True)
df_closings.fillna(value=0, inplace=True)       



In [ ]:
import warnings
warnings.filterwarnings('ignore')

def cointegrated_pairs(data):
    n =  (len(data.columns) -1)
    stocks = data.iloc[:,1:].columns.values #pulls all stock names 
    data = data.to_numpy()
    pvalue_matrix = np.zeros((n,n),dtype = float)
    pairs = np.empty((n*n,3),dtype=object)
    m = 0
    # loop through all possilbe combintions of stocks ie 3!=6
    for i in range(0,n):
        for j in range(0,n):
            S1 = data[:,i+1]
            S2 = data[:,j+1]
            result = ts.coint(S1,S2)
            pvalue_matrix[i,j] = result[1]
            #storing all paris in matrix form
            if (result[1] < 0.05) & (i!=j):
                pairs[m,0] = stocks[i]
                pairs[m,1] = stocks[j] 
                pairs[m,2] = result[1]
                m = m + 1
    try:
        a_file = open("pairs_temp_%s.txt" % str(datetime.utcnow().replace(microsecond=0).isoformat()), "w")
        for row in pairs:
            if row[0] is not None:
                a_file.write("%s/%s: %s\n" % (row[0], row[1], row[2]))
        a_file.close()
    except:
       print("Couldn't save pairs to txt file") 
       
    return pvalue_matrix,  pairs

In [ ]:
# on the MBP M1 it is ~ 37s per 100 items (10x10 np array)
# checking 1600 instruments would take ofer 263h (~ 11 days)
pvalue_matrix,  pairs = cointegrated_pairs(df_closings)